### Краткая формулировка задачи: необходимо предсказать по слову, является ли оно фамилией (Принадлежит ли к классу 1 или 0)

### Итоговое решение оказалось простым - логистическая регрессия. Признаки получены из CountVectorizer по n-граммам из слова, n = 1,...,7.
### При этом пытаясь улучшить результат, я использовал и более сложные модели. Пробовал делать блендинг с линейными моделями и деревьями; строить рандомные леса; добавлять к уже полученным признакам какие-то свои (пришлось повозиться с добавлением чего-либо к разреженной матрице); пробовал брать только часть слов, так как было подозрение, что окончания должны больше определять, является ли слово фамилией; пробовал модели с изменением ядра. Во всех перечисленных способах менял параметры моделей. Но все это давало результат хуже :(
### В качестве ответов лучше давать вероятности принадлежности классу, так как метрика roc-auc. Кросс-валидация это уже учитывает, поэтому проверка на ней давала достаточно достоверные результаты

In [74]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, RandomizedLogisticRegression
from sklearn.metrics import mean_squared_error, roc_auc_score
import sklearn.linear_model
from sklearn import ensemble
from sklearn import svm
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer, FeatureHasher
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MaxAbsScaler, PolynomialFeatures
from scipy import optimize
from sklearn import model_selection
from sklearn import datasets, cross_validation, metrics, neighbors
from sklearn.model_selection import StratifiedKFold
from matplotlib.colors import ListedColormap
from scipy.sparse import dok_matrix
from pandas import DataFrame
import xgboost
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [62]:
train = []
test = []
sample_submission = []
with open("linear_train.txt") as file:
    for line in file:
        string = line.split(',')
        train.append((string[0], float(string[1])))
with open("linear_test.txt") as file:
    for line in file:
        test.append(line.split('\n')[0])
sample_submission = pd.read_csv("linear_ans_example.txt")

In [63]:
#test = pd.DataFrame(test)
#train = pd.DataFrame(train)
#sample_submission = pd.DataFrame(sample_submission)

In [64]:
train_target = []
train_data = []
for i in range(len(train)):
    train_target.append(train[i][1])
    train_data.append(train[i][0])

In [189]:
count_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(1, 7), lowercase=False)
count_vectorizer.fit(train_data)
sparse_feature_matrix = count_vectorizer.transform(train_data)
sparse_feature_matrix_test = count_vectorizer.transform(test)
model = LogisticRegression(dual=True)
model.fit_transform(sparse_feature_matrix, train_target)
preds = model.predict_proba(sparse_feature_matrix_test)

/home/evgeny/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


In [194]:
pipeline = make_pipeline(CountVectorizer(analyzer='char_wb', ngram_range=(1, 7), lowercase=False),
                         MaxAbsScaler(),
                         LogisticRegression(dual=True, n_jobs=-1))
pipeline.fit(train_data, train_target)
preds = pipeline.predict_proba(test)

In [195]:
len(preds)

188920

In [196]:
preds

array([[  7.74798181e-01,   2.25201819e-01],
       [  8.47982539e-01,   1.52017461e-01],
       [  8.97636717e-01,   1.02363283e-01],
       ..., 
       [  9.86008220e-01,   1.39917800e-02],
       [  9.99828733e-01,   1.71266721e-04],
       [  9.99854986e-01,   1.45014379e-04]])

In [197]:
y = []
for i in range(len(preds)):
    y.append(preds[i][0])

In [198]:
len(y)

188920

In [199]:
y

[0.77479818059688399,
 0.84798253888819064,
 0.89763671685299184,
 0.96576284612576346,
 0.89538944738248905,
 0.91886448086728501,
 0.91392544901212291,
 0.91419619395562923,
 0.91419619395562923,
 0.91747237985426355,
 0.91747237985426355,
 0.94148557057210303,
 0.97432646843888226,
 0.96375967787626005,
 0.9177068510971661,
 0.88289766629262956,
 0.83667209359529693,
 0.84538738867885255,
 0.050701376164331013,
 0.0072486285260492389,
 0.0030491285345743968,
 0.94846152948177664,
 0.99901144636907546,
 0.99990269500594664,
 0.99987596832587156,
 0.99938326098577179,
 0.99532388583106857,
 0.99897381857587231,
 0.30966441277589007,
 0.93923282379741291,
 0.99019933251076653,
 0.60112452391083893,
 0.83730641638582537,
 0.90209263655729943,
 0.99771526204675109,
 0.99728651789693923,
 0.80529858300813006,
 0.99577509185213431,
 0.73127809130398491,
 0.99099231639312491,
 0.93962719552421481,
 0.0021723495991511355,
 0.63021498921370267,
 0.0039484154151073758,
 0.0015248336018387487,


In [200]:
sample_submission = pd.DataFrame(sample_submission)

In [201]:
sample_submission['Answer'] = y

In [202]:
sample_submission.to_csv("linear_ans_3.tsv", sep=',', index=False)